# Reshaping of data and CSV exports

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno

In [ ]:
# load the data
data = pd.read_csv('data/Train.csv')

In [ ]:
data.head(1)

In [ ]:
# retyping strings to lists for pd.explode() to work
for c in data.columns[2:-1]:
    data[c] = data[c].apply(lambda x: [a for a in str(x).split(",")])

In [ ]:
data.head(1)

In [ ]:
# generating hours column counting from 0 to 120 (5 days with first day having 25 measurements)
hours = list(range(121))

# inserting column before target column
data.insert(len(data.columns)-1, 'hours', [hours] * data.shape[0])

In [ ]:
# generating days column counting from 1 to 5 (first day having 25 measurements)
day = [1, 2, 3, 4, 5]
rep = [25, 24, 24, 24, 24]
days = np.repeat(day, rep)

# inserting column before target column
data.insert(len(data.columns)-1, 'day', [days] * data.shape[0])

In [ ]:
data.head(1)

In [ ]:
# Explode the dataframe to convert from pseudo-wide to long format
data = data.explode(data.columns[2:-1].to_list())

In [ ]:
# retype all values from string (was necessary for split) to float
for c in data.columns[2:]:
    data[c] = data[c].apply(lambda x: float(x))

In [ ]:
data.info()

In [ ]:
# export the long dataframe as csv
data.to_csv('data/train_long.csv')

In [ ]:
# to further work with the data, replace all 'nan' strings with np.nan
data = data.replace('nan', np.nan)

In [ ]:
# calculate how many observations have missing data (%)
np.round(data.isna().sum()/data.shape[0] * 100, 2)

In [ ]:
# check overlap of missing data
msno.matrix(data, color=(0, 0, 0.33));

In [ ]:
data.head(10)

In [ ]:
def aggregate(data, group: list, cid: tuple, func: str) -> pd.DataFrame:
    """Returns one of [mean, median, min, max] for dataframe grouped by 'group'.

    Args:
        data (pd.DataFrame): _description_
        group (list): column names by which to group by
        cid (list): boundary column indices (left, right)
        func (str): accepts mean, median, min, max

    Returns:
        pd.DataFrame: 
    """
    if func == 'mean':
        return data.groupby(group)[data.columns[cid[0]:cid[1]]].mean()
    elif func == 'median':
        return data.groupby(group)[data.columns[cid[0]:cid[1]]].median()
    elif func == 'min':
        return data.groupby(group)[data.columns[cid[0]:cid[1]]].min()
    elif func == 'max':
        return data.groupby(group)[data.columns[cid[0]:cid[1]]].max()

In [ ]:
# get different aggregates from the data and export as csv
for i in ['mean', 'median', 'min', 'max']:
    aggregate(data, ['ID', 'day'], (2,-3), i).to_csv(f'data/train_daily_{i}.csv')